# BigTransfer (BiT) Fine-Tuning

- 参考資料  
  https://blog.tensorflow.org/2020/05/bigtransfer-bit-state-of-art-transfer-learning-computer-vision.html

以下メモ

## BiT-HyperRule

BiTをファインチューニングするために用意されているヒューリスティックな方法。  
Hyper-parameterをランダム探索した方が、最適にはなるが非常にコストがかかるので、「この設定でやると、1回のファインチューニングでイイ感じになるよ」というもの。

- 使用最適化アルゴリズム: SGD
  - Learning rate: 0.003
  - Momentum: 0.9
  - 備考  
  学習ステップが、30%, 60%, 90% になるタイミングで、学習率を$\frac{1}{10}$ずつ減衰させる。

> we decay the learning rate by a factor of 10 at 30%, 60% and 90% of the training steps.

とあるけど、参考元のColabだと $\frac{1}{10}, \frac{1}{10}, ...$ と減衰させてない・・・ 

- バッチ数: 512  
  → 512とかメモリ的に無理って人向けに、バッチサイズを小さくした際の処理（Learning Rate, step数の調整）がある。

- 学習用データへの前処理
  - リサイズ、ランダムクリップ、水平方向へのランダムフリップを行う。リサイズする画像サイズは、参考資料のTable 1。
  - タスクの種類によっては行わない方がいい処理アリ。（正解ラベルと乖離が出てしまう）
    - 物体の数え上げ ⇒ ランダムクリップはNG。
    - 物体の位置特定 ⇒ ランダムフリップはNG。
  - MixUpの利用  
    使用するデータセットで、使うか否かの判断基準あり。

- 学習ステップ  
  データセットのサイズで決定される。

- その他
  - データセットの画像サイズでリサイズ指定あるけど、96px以下がたまに含まれてるとかある場合どうするんだろうか？  
    今回は、フィルターして除いてしまう。

In [1]:
# -*- coding: utf-8 -*-
import dataclasses
import logging
import pathlib
import datetime
import json
import itertools
import random

import numpy as np
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds


class WrappedBiT(tf.keras.Model):

    def __init__(self, classes=2, bit_type='m-r50x1', version='1'):
        super().__init__()
        self._head = tf.keras.layers.Dense(classes, kernel_initializer='zeros')
        self._bit = tfhub.KerasLayer(f'https://tfhub.dev/google/bit/{bit_type}/{version}')

    def call(self, inputs, training=None, mask=None):
        features = self._bit(inputs)
        return self._head(features)


@dataclasses.dataclass
class HyperRule:
    steps: int
    resize_edge: int
    crop_size: int
    optimizer: tf.keras.optimizers.Optimizer

    def __init__(self, image_edge: int, dataset_size: int, batch_size=512):
        assert 512 % batch_size == 0
        self.batch_size = batch_size

        if image_edge < 96:
            self.resize_edge, self.crop_size = 160, 128
        else:
            self.resize_edge, self.crop_size = 512, 480

        if dataset_size < 20 * 10 ** 3:
            steps, boundaries = 500, [200, 300, 400]
        elif 20 * 10 ** 3 <= dataset_size < 500 * 10 ** 3:
            steps, boundaries = 10000, [3000, 6000, 9000]
        else:
            steps, boundaries = 20000, [6000, 12000, 18000]
        self.steps = steps * 512 // batch_size

        lr = 0.003 * batch_size / 512
        lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            boundaries=boundaries, values=[lr, lr * 1e-1, lr * 1e-2, lr * 1e-3]
        )
        self.optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9
        )

- データセットの準備

`dataset_cats_and_dogs`

今回は犬猫を選択。手元にラベル付きデータの準備がないとかなら、Tensorflow Datasetsから拝借できる。  
ここでは、一辺96px以下になるような画像はフィルターし、残った全体の80%を学習、20%をバリデーションに使う。

`setup_dataset_and_hyperrule`

データセットにHyperRuleを適応しつつ、キャッシュとか順序シャッフルもセットアップ。

In [2]:
def dataset_cats_and_dogs():
    def filter_small(image, _):
        return tf.reduce_all(tf.shape(image)[:2] > tf.constant(96))

    ds_org: tf.data.Dataset
    (ds_org, ), info = tfds.load(name='cats_vs_dogs', with_info=True, as_supervised=True, split=['train'])
    ds_filtered = ds_org.filter(filter_small)

    # filter smaller than 96x96
    num_samples = ds_filtered.reduce(np.int64(0), lambda x, _: x + 1).numpy()
    num_samples_train = int(num_samples * 0.8)
    ds_train = ds_filtered.take(num_samples_train)
    ds_test = ds_filtered.skip(num_samples_train)

    label_names = info.features['label'].names
    return (ds_train, num_samples_train), (ds_test, num_samples - num_samples_train), label_names, 97


def setup_dataset_and_hyperrule(ds_factory, cache_dir, buffer_size=None, batch_size=512):
    def resize_normalize(image, label):
        image = tf.image.resize(image, [rule.resize_edge, rule.resize_edge],
                                method=tf.image.ResizeMethod.BILINEAR)
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    def random_sampling(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_crop(image, [rule.crop_size, rule.crop_size, 3])
        return image, label

    (ds_train, ds_train_size), (ds_test, _), label_names, image_edge = ds_factory()

    rule = HyperRule(image_edge=image_edge, dataset_size=ds_train_size, batch_size=batch_size)
    ds_train = (ds_train
                .map(resize_normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .cache(str(cache_dir / 'train'))
                .shuffle(buffer_size or ds_train_size)
                .repeat()
                .map(random_sampling, num_parallel_calls=tf.data.experimental.AUTOTUNE)
                .batch(rule.batch_size)
                .prefetch(tf.data.experimental.AUTOTUNE))
    ds_test = (ds_test
               .map(resize_normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
               .cache(str(cache_dir / 'test'))
               .batch(rule.batch_size)
               .prefetch(tf.data.experimental.AUTOTUNE))

    return rule, ds_train, ds_test, label_names

`dataset_mydataset`

自分でデータ持ってる用。小さい画像(96px以下)をフィルターして、全体の90%を学習、10%をバリデーションに使う。シャッフルした後に分割する。  
パス情報なら、メモリへの負担は比較的マシやろと乱暴に全体シャッフルしてる。あんまよくないと思う。。

In [3]:
def dataset_mydataset():
    def filter_small(image, _):
        return tf.reduce_all(tf.shape(image)[:2] > tf.constant(96))

    def generator():
        path_iter = itertools.chain(pathlib.Path('train/cat').iterdir(), pathlib.Path('train/dog').iterdir())
        path_list = list(path_iter)
        for path in filter(lambda p: p.suffix in {'.png', '.jpg'}, random.sample(path_list, len(path_list))):
            try:
                image = tf.image.decode_image(tf.io.read_file(str(path)))
            except Exception:
                continue
            yield image, tf.constant(label_str2int[path.parents[0].stem])

    label_str2int = {'cat': 0, 'dog': 1}
    ds_org = tf.data.Dataset.from_generator(generator,
                                            (tf.float32, tf.int32),
                                            (tf.TensorShape([None, None, 3]), tf.TensorShape([])))
    ds_filtered = ds_org.filter(filter_small)

    # filter smaller than 96x96
    num_samples = ds_filtered.reduce(np.int64(0), lambda x, _: x + 1).numpy()
    num_samples_train = int(num_samples * 0.9)
    ds_train = ds_filtered.take(num_samples_train)
    ds_test = ds_filtered.skip(num_samples_train)

    return (ds_train, num_samples_train), (ds_test, num_samples - num_samples_train), ['cat', 'dog'], 97

- 学習の準備

自分のメモリ容量に応じて、`buffer_size`, `batch_size` 調整して学習始める。

- epochsはコメントアウトしてある回数を指定する。
  - 最初はテストで、10回くらいとかでいい。
- ルールで回数が指定されているので回しきる。EarlyStoppingは使わない。

In [4]:
def train(bit_model: WrappedBiT, dst_dir: pathlib.Path, dataset_factory, cache_dir, buffer_size=None, batch_size=512):
    rule, ds_train, ds_test, label_names = setup_dataset_and_hyperrule(
        dataset_factory, cache_dir, buffer_size=buffer_size, batch_size=batch_size
    )
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    bit_model.compile(optimizer=rule.optimizer, loss=loss_fn, metrics=['accuracy'])

    callbacks = [
        # tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=str(dst_dir / 'tmp.ckpt'), verbose=1, save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir=str(dst_dir / 'tfboard'), histogram_freq=1, write_images=1),
    ]

    bit_model.fit(
        ds_train,
        batch_size=rule.batch_size,
        steps_per_epoch=10,
        epochs=10, # rule.steps // 10,
        validation_data=ds_test,
        callbacks=callbacks
    )
    bit_model.save(str(dst_dir / 'model'), save_format='tf')
    (dst_dir / 'label_names.json').write_text(json.dumps(label_names, ensure_ascii=False), encoding='utf-8')


def main():
    tf.get_logger().setLevel(logging.ERROR)
    for d in tf.config.experimental.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(d, True)

    bit_type = 'm-r50x1'
    model = WrappedBiT(classes=2, bit_type=bit_type)

    version = '1'
    ds_factory = dataset_mydataset

    model_dir = pathlib.Path(f'models/{bit_type}/{version}')
    cache_dir = pathlib.Path('cache/cats_dogs')
    model_dir.mkdir(exist_ok=False, parents=True)
    cache_dir.mkdir(exist_ok=True, parents=True)

    train(model, model_dir, ds_factory, cache_dir, buffer_size=2000, batch_size=64)

In [6]:
main()

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.2726 - accuracy: 0.8781
Epoch 00001: val_loss improved from inf to 0.03761, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 31s 3s/step - loss: 0.2726 - accuracy: 0.8781 - val_loss: 0.0376 - val_accuracy: 0.9899
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 0.0271 - accuracy: 0.9922
Epoch 00002: val_loss improved from 0.03761 to 0.02982, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 28s 3s/step - loss: 0.0271 - accuracy: 0.9922 - val_loss: 0.0298 - val_accuracy: 0.9937
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 0.0147 - accuracy: 0.9969
Epoch 00003: val_loss improved from 0.02982 to 0.01091, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 28s 3s/step - loss: 0.0147 - accuracy: 0.9969 - val_loss: 0.0109 - val_accuracy: 0.9975
Epoch 4/10
10/10 [=======================